In [ ]:
"""
A collection of functions to perform a dark photon search on an averaged spectrum


Functions
-------
median_filt(
    
):
    LPF for spectrum. Recomended as a pre-filter if
    spec contains large, very narrow candidates

butter_filt(

):
    LPF for spectrum. 

SG_filt(

):
    LPF for spectrum

match_filt(
    
):
    Performs a matched filtering on approx. normally distributed data
    Internal helper functions include freq2idx and genNuSteps

set_limit(

):
    Sets a limit (or series of limits) on an array of 
    normally distributed data (or list of arrays of normal data)

get_required_sigma(

):
    Given a number of samples and a desired confidence limit, tells you how 
    many "N" sigma you need to go to in order to be CL% confident no samples
    are above the calculated "N" 
"""
import numpy as np 
import scipy.signal as spsig
import scipy.ndimage as spim
import cupyx.scipy.ndimage as cpim
import cupy as cp
from scipy.signal import butter, filtfilt
import scipy.stats
import scipy.special
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
%matplotlib widget


def median_filt(spec, filter_size=50, gpuFlag = 1):
    """
    Function to apply median filtering to a spectrum.

    Parameters
    ----------
    spectrum : array
        The input power spectrum.

    filter_size : int, optional
        Size of the median filter. Default is 50.

    Returns
    -------
    spectrum_filtered : array
        The median-filtered spectrum.
    """
    if gpuFlag:
        spectrum_gpu = cp.array(spec)
        spectrum_filtered_gpu = cpim.median_filter(
            spectrum_gpu,
            size=(filter_size),
            origin=0
        )
        spectrum_filtered = spectrum_filtered_gpu.get()
    else:
        spectrum_filtered = spim.median_filter(
            spec,
            size=(filter_size),
            origin=0
        )
    return spectrum_filtered

def butter_filt(spec, cutoff=40000, order=6):
    """
    Function to apply Butterworth low-pass filtering to a spectrum.

    Parameters
    ----------
    spectrum : array
        The input power spectrum.

    cutoff : int, optional
        Cutoff frequency for the filter. Default is 40000.

    order : int, optional
        Order of the filter. Default is 6.

    Returns
    -------
    spectrum_filtered : array
        The Butterworth low-pass filtered spectrum.
    """
    nyq = 0.5 * len(spec)  # Nyquist frequency
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    spectrum_filtered = filtfilt(b, a, spec)
    return spectrum_filtered

def sg_filt(spec, window_length=51, polyorder=4):
    """
    Apply a Savitzky-Golay filter to the spectrum.

    Parameters
    ----------
    window_length : int, optional
        Length of the filter window; must be a positive odd integer, default is 51.
    polyorder : int, optional
        Order of the polynomial used to fit the samples, default is 1.

    Returns
    -------
    arr
        Filtered spectrum.
    """
    spectrum_filtered = spsig.savgol_filter(spec, window_length=window_length, polyorder=polyorder)
    return spectrum_filtered

def matched_filt(freqs, 
    spec, 
    startFreq, 
    stopFreq, 
    percent, 
    overlap
    ):
    """
    Apply a matched filter to the input spectrum over a specified frequency range.

    This function divides the freqs range into steps and applies a convolution
    operation with a lambda template for each step. The function returns the 
    convolved output for each freqs segment.

    Parameters
    ----------
    freqs : array-like
        Array of frequencies corresponding to the input spectrum.
    spec : array-like
        Array of spectral values to be filtered.
    startFreq : float
        Starting freqs of the range over which the filter is applied.
    stopFreq : float
        Stopping frequency of the range over which the filter is applied.
    percent : float
        Percentage to determine the step size between successive frequencies in the
        range (startFreq to stopFreq).
    overlap : int
        Number of overlapping elements between successive frequency segments.

    Returns
    -------
    tuple
        A tuple containing two lists:
        - outputFreqs: List of arrays with the frequencies corresponding to each segment.
        - outputConv: List of arrays with the convolved values for each segment.

    Notes
    -----
    Frequency step generation (`gen_nu_steps`) is
    defined internally within this function.
    """
    def gen_nu_steps(freqs, startFreq, stopFreq, percent):
        freq    = startFreq
        nuStepList  = []
        while freq < stopFreq:
            #if freq > freqs:
            #    nu0Arr.append(400)
            nuStepList.append(freq)
            endFreq = freq + freq*percent/100
            freq = endFreq
        nuStepList.append(stopFreq) 
        return nuStepList
        
    nuStepList  = gen_nu_steps(freqs, startFreq, stopFreq, percent)
    outputConv  = []
    outputFreqs = []
    #change template over frequency range
    for i, nu in enumerate(nuStepList[0:-1]):
        idxStart        = freq2idx(nuStepList[i], freqs) - overlap
        nu0             = nuStepList[i] + (nuStepList[i+1] - nuStepList[i])/2
        if 0: #nuStepList[i+1] == stopFreq:
            idxStop     = freq2idx(nuStepList[i+1], freqs) - 10
        else:
            idxStop     = freq2idx(nuStepList[i+1], freqs) + overlap
        lamOut      = lam(freqs, nu0, p0=1, nBins=100)
        subConv     = np.convolve(np.flip(lamOut[1]), spec[idxStart:idxStop], mode='same')[overlap:-overlap]
        subFreqs    = freqs[idxStart+overlap:idxStop-overlap]
        
        outputConv.append(subConv)
        outputFreqs.append(subFreqs)
    return (outputFreqs, outputConv)

def lam(nu, nu0, p0 = None, nBins=None, v0=220, vlab=233, offset = 0):
    """
    Lambda function for matched filtering.
    Work in MHz, this will explode for large frequency values (ie 100e6 Hz)
    velocity args in km/s
    vlab has yearly modulation between about 220 and 255, see fig 3a in Gramolin
    output is normilized to p0 if given

    Parameters
    ----------
    nu : float
        Frequency.
    nu0 : float
        Center frequency.
    p0 : float, optional
        Power normalization value, default is None.
    nBins : int, optional
        Number of bins, default is None.
    v0 : int, optional
        Velocity parameter, default is 220.
    vlab : int, optional
        Lab velocity parameter, default is 233.
    offset : int, optional
        Frequency offset, default is 0.

    Returns
    -------
    tuple
        Frequencies and output power spectral density.
    """
    if nBins is not None:
        # find the index of nu0 in nu array
        idx = np.abs(nu-nu0).argmin()

        # calculate the slice start and end indices
        start_idx = max(0, idx - nBins//2)
        end_idx = min(len(nu), idx + nBins//2)
        nu = nu[start_idx:end_idx]

    c       = 299792 #km/s
    nu0     = nu0 + offset
    beta    = np.zeros_like(nu) # initialise beta with zeros

    mask    = (nu > nu0) # create a mask of values where nu > nu0
    # calculate beta only for values where nu > nu0
    beta[mask] = (2*c*vlab)/(v0**2) * np.sqrt((2*(nu[mask]-nu0))/(nu0))
    beta    = np.minimum(beta, 100)
    a       = (2*c**2)/(np.sqrt(np.pi)*v0*vlab*nu0)
    b       = -((beta**2*v0**2)/(4*vlab**2)) - (vlab**2/v0**2) 

    lamArr  = a*np.exp(b)*np.sinh(beta)
    if p0 is not None:
        outPSD = p0/np.sum(lamArr) * lamArr #normilize to p0
    else:
        outPSD = lamArr

    return nu, outPSD 

def get_required_sigma(nBins, desired_confidence):    
    """
    Calculate the required standard deviation (sigma) level to achieve a specified
    confidence level that none of the samples in a given number of bins will exceed
    that sigma level.

    The function solves for the sigma level using the cumulative distribution function
    of the normal distribution and the binomial probability for the given number of bins.

    Parameters
    ----------
    nBins : int
        The number of bins or samples drawn from a normal distribution.
    desired_confidence : float
        The desired confidence level (between 0 and 1) for the probability that none
        of the samples will exceed the calculated sigma level. eg 0.95

    Returns
    -------
    float
        The calculated sigma level required to achieve the desired confidence.

    Notes
    -----
    The function uses the `fsolve` method from SciPy's optimize module to solve for
    the sigma value. If the solver does not converge, it returns a guessed value
    based on the inverse error function.
    """

    desired_confidence = 1-desired_confidence
    def probability_greater_than(sigma):
        return 0.5-scipy.special.erf(sigma/np.sqrt(2))/2

    def binomial(N, num_successes, prob_of_success):
        return scipy.special.comb(N, num_successes)*prob_of_success**(num_successes)*(1-prob_of_success)**(N-num_successes)

    def probability_none_above(nBins, sigma):
        # returns probability that after nBins of draws from a normal distribution
        # that none of the draws will be above the sigma upper limit (mean+sigma)
        return 1-binomial(nBins, nBins, 1-probability_greater_than(sigma))

    guess     = scipy.special.erfinv(1-(2*(desired_confidence)/nBins))*np.sqrt(2) #from taylor expantion
    nSig, solDict, itr, msg   = fsolve(lambda sigma: probability_none_above(nBins, sigma)-desired_confidence, guess, full_output=True)
    nSig = nSig[0]
    if itr != 1:
        print('fSolve in get_required_sigma failed. Using guess')
        print(msg)
        nSig = guess
    
    return nSig

def set_lim(freqs, spec, CL):
    """
    Calculate the frequency limits and standard deviations for the given spectra.

    This function computes the limits by applying the required sigma to each segment
    of the input spectra and then concatenates the results to form the complete limit.
    It also flattens and concatenates the input frequency and spectrum arrays to match
    the shape of the limit.

    Parameters
    ----------
    freqs : arr or list of arr
        Array(s) of frequencies corresponding to the input spectra. If a single array
        is provided, it is wrapped in a list to form an array of arrays.
    spec : arr or list of arr
        Array(s) of spectra to calculate the limits for. If a single array is provided,
        it is wrapped in a list to form an array of arrays.
    CL : float
        Confidence level for calculating the required sigma. This value is used to
        determine how many standard deviations to apply to the spectra.

    Returns
    -------
    tuple
        A tuple containing three arrays: the concatenated frequency array, the
        concatenated spectrum array, and the calculated limit array.
    """
    # If the inputs are 1-D arrays, wrap them in a list to make them an array of arrays
    if isinstance(spec, (np.ndarray, list)) and not isinstance(spec[0], (np.ndarray, list)):
        freqs   = [freqs]
        spec    = [spec]
    nSamps = len(np.hstack(freqs))
    nSig = get_required_sigma(nSamps, CL)

    limits = []
    for idx in range(len(spec)):
        limits.append(nSig * 1.4826 * scipy.stats.median_abs_deviation(spec[idx]))

    limit   = np.concatenate([np.repeat(limit, len(freqs)) for limit, freqs in zip(limits, freqs)])
    freqs   = np.hstack(freqs)
    spec    = np.hstack(spec)

    return (freqs, spec, limit)

def freq2idx(freq, freqs):
    return int(freq/freqs[-1]*len(freqs))


In [ ]:
# analysis function

def analysis(freqs,
    spec,
    freqStart,
    freqStop,
    CL,
    medFiltBins,
    butterFreq,
    butterOrder,
    percent,
    overlap,
    stepsize = 1000e-6
    ):
    idxStart    = freq2idx(freqStart, freqs)
    idxStop     = freq2idx(freqStop, freqs)
    
    # Fit H to spec
    medianFilt = median_filt(spec, filter_size=medFiltBins)
    H = (butter_filt(medianFilt, cutoff = butterFreq, order = butterOrder))

    #Norm spec: mean = 1, STD = 1
    normSpec    = spec / H

    nSigmaTup   = set_lim(freqs, normSpec, 0.95)
    nSigmaLim   = nSigmaTup[2]

    nSigma      = get_required_sigma((idxStop - idxStart), 0.95)
    oneSigMad   = (1.4826*scipy.stats.median_abs_deviation(normSpec))


    
    matchedOut      = matched_filt(freqs=freqs,
        spec=normSpec,
        startFreq=freqStart,
        stopFreq=freqStop,
        percent=percent,
        overlap=overlap)
    matchedFreqs    = matchedOut[0]
    matchedSpec     = matchedOut[1]

    nSigmaTup       = set_lim(matchedFreqs, matchedSpec, CL)
    matchedFreqs    = nSigmaTup[0]
    matchedSpec     = nSigmaTup[1]
    matchedLim      = nSigmaTup[2]

    # sloppy print t-sigma of candidate
    canIdx = (np.where(((matchedSpec - 1) > matchedLim) == True)) 
    print(f't-sigma: {matchedSpec[canIdx]/(1-matchedLim[canIdx])}')
    
    rawCanList  = freqs[idxStart:idxStop][((matchedSpec - 1) > matchedLim)]
    rawCanList = np.asarray(rawCanList)
    splitList =  np.split(rawCanList, np.where(np.diff(rawCanList) > stepsize)[0]+1)
    if len(rawCanList) == 0:
        return(np.asarray([]))
    canList = [can[0] for can in splitList]

    return canList

In [ ]:
#analysis params
medFiltBins = 51
butterFreq  = 12000
butterOrder = 6
startFreq   = 250    # MHz
stopFreq    = 1050   # MHz
percent     = 25    # matched filter
overlap     = 100   # matched filter, bins
CL          = 0.95  # confidence limit

freqs = np.linspace(0, 1200, 2**22)
spec = np.load('./data_gitignore/12hrRun_4dBatt_9_6_23.npy', allow_pickle = True).item()['chBSpec_W']


canList = analysis(freqs,spec,medFiltBins=medFiltBins,butterFreq=butterFreq,butterOrder=butterOrder,freqStart=startFreq,freqStop=stopFreq,percent=percent,overlap=overlap,CL = CL)

In [ ]:
canList

In [ ]:
freqs = np.linspace(0, 1200, 2**22)
spec = np.load('./data_gitignore/30minuteRun_4dBAtt_9_6_23.npy', allow_pickle = True).item()['chBSpec_W']
specVeto = np.load('./data_gitignore/30minuteRun_4dBAtt_9_6_23.npy', allow_pickle = True).item()['chASpec_W']

#analysis params
medFiltBins = 51
butterFreq  = 12000
butterOrder = 6
startFreq   = 400    # MHz
stopFreq    = 700   # MHz
percent     = 50    # matched filter
overlap     = 100   # matched filter, bins
CL          = 0.95  # confidence limit

# Fit H to spec
#medianFilt = median_filt(spec, filter_size=medFiltBins)
H = (butter_filt(spec, cutoff = butterFreq, order = butterOrder))

#Norm spec: mean = 1, STD = 1
normSpec    = spec / H

plt.close('all')

plt.figure()
plt.plot(freqs[:-100], spec[1:-100])
plt.plot(freqs[:-100], H[1:-100])



plt.figure()
plt.plot(freqs[10000:-100],normSpec[10001:-100])

plt.figure()
plt.plot(freqs, 10*np.log10(specVeto[1:]*1000))

In [ ]:
freqs = np.linspace(0, 1200, 2**22)
fileName        = '12hrRun_4dBatt_9_6_23.npy'
spec = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']
specVeto = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chASpec_W']

#analysis params
medFiltBins = 51
butterFreq  = 12000
butterOrder = 6
startFreq   = 400    # MHz
stopFreq    = 700   # MHz
percent     = 50    # matched filter
overlap     = 100   # matched filter, bins
CL          = 0.95  # confidence limit

# Fit H to spec
#medianFilt = median_filt(spec, filter_size=medFiltBins)
H = (butter_filt(spec, cutoff = butterFreq, order = butterOrder))

#Norm spec: mean = 1, STD = 1
normSpec    = spec / H

plt.close('all')

plt.figure()
plt.plot(freqs[:-100], 10*np.log10(1000*spec[1:-100]))
#plt.plot(freqs[:-100], H[1:-100])
plt.grid(True)


plt.figure()
plt.title('Normalized Spectrum')
plt.plot(freqs[10000:-100],normSpec[10001:-100])
plt.xlabel('Frequency (MHz)')
plt.ylabel('Power (Linear Ratio)')

plt.figure()
plt.plot(freqs, 10*np.log10(specVeto[1:]*1000))

In [ ]:
#this is really thermal noise stuff and should go into it's own notebook
#10k averages
freqs = np.linspace(0, 1200, 2**22)
startFreq   = 950 #MHz
stopFreq    = 1050 #MHz

startIdx    = int(startFreq/1200*2**22)
stopIdx     = int(stopFreq/1200*2**22)

'''
#scan
fileName        = 'antSpec1.npy'
spec1 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#bump dipole antenna 1/4 inch
fileName        = 'antSpec2.npy'
spec2 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#open and close door
fileName        = 'antSpec3.npy'
spec3 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#turn attenuator from 0 to 30dB
fileName        = 'antSpec4.npy'
spec4 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

# put pcb vivaldi on box
fileName        = 'antSpec5.npy'
spec5 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

# terminate vivaldi
fileName        = 'antSpec6.npy'
spec6 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

# touch vivaldi
fileName        = 'antSpec7.npy'
spec7 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

# bump vivaldi ~1mm (for real, as small as I could)
fileName        = 'antSpec8.npy'
spec8 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

# termanator off vivaldi, didn't move
fileName        = 'antSpec9.npy'
spec9 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

# put pcb vivaldi behind big vivaldi facing away. Terminator off
fileName        = 'antSpec10.npy'
spec10 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

# Terminate vivaldi
fileName        = 'antSpec11.npy'
spec11 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

# switch to bicon
fileName        = 'antSpec12.npy'
spec12 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#yagi
fileName        = 'antSpec13.npy'
spec13 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#terminator
fileName        = 'antSpec14.npy'
spec14 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#open
fileName        = 'antSpec15.npy'
spec15 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#short
fileName        = 'antSpec16.npy'
spec16 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#term into 470MHz LPF
fileName        = 'antSpec17.npy'
spec17 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#short into double turn around (basically ~1 inch cable)
fileName        = 'antSpec18.npy'
spec18 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#open into double turn around (basically ~1 inch cable)
fileName        = 'antSpec19.npy'
spec19 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

### Doesnt seem like the turnaround matters much

# term into 470MHz LPF into 1dB attenuator
fileName        = 'antSpec20.npy'
spec20 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

# term into 470MHz LPF into 2 * 1dB attenuator
fileName        = 'antSpec21.npy'
spec21 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

### 9/8/23

# big vivaldi
fileName        = 'antSpec22.npy'
spec22 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

# put open 50cm (total) dipole about 2 ft in front of vivaldi 
fileName        = 'antSpec23.npy'
spec23 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

# terminate dipole
fileName        = 'antSpec24.npy'
spec24 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

### 9/13/23

#vivaldi looking into corner
# should see its reflection
fileName        = 'antSpec25.npy'
spec25 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#vivaldi looking across wall
# should only see its reflecton after many bounces
fileName        = 'antSpec26.npy'
spec26 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#same vivaldi looking across wall but add foam
# should only see its reflecton after many bounces down by foam
fileName        = 'antSpec27.npy'
spec27 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']
'''
#### 9/27/23
#vivaldi looking into corner, run 1Bp1 amps, sph1200+ LPF, 10k avg, 2^20pt fft 
freqs = np.linspace(0, 1250, 2**17)

#recieve vivaldi, 50cm dipole open
fileName        = 'antSpec28.npy'
spec28 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#recieve vivaldi, 50cm dipole term
fileName        = 'antSpec29.npy'
spec29 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#terminator + 1dB att
fileName        = 'antSpec30.npy'
spec30 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#terminator np att. 
#no difference.
fileName        = 'antSpec31.npy'
spec31 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#open
fileName        = 'antSpec32.npy'
spec32 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#short
fileName        = 'antSpec33.npy'
spec33 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#circulator. port 1,2,3 = short, chode cable, terminator.
fileName        = 'antSpec34.npy'
spec34  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']


#circulator. port 1,2,3 = short, chode cable, open.
fileName        = 'antSpec35.npy'
spec35  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#hang citculator on amp. Remove swich (was used in 28-35)
#circulator. port 1,2,3 = chode (short on other end), amp , term.
fileName        = 'antSpec36.npy'
spec36  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#circulator. port 1,2,3 = chode (vivaldi on other end), amp , term.
fileName        = 'antSpec37.npy'
spec37  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#same as 37, nudge dipole
fileName        = 'antSpec38.npy'
spec38  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#same as 37/38 but connect port 3 of circulator to term on patch pannel
fileName        = 'antSpec39.npy'
spec39  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#same as above, open on port 3 via patch pannel
fileName        = 'antSpec40.npy'
spec40  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#same as above, term on port 3 via patch pannel WARM FOR 1 MIN WITH HEAT GUN
fileName        = 'antSpec41.npy'
spec41  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#let term cool for 10 minutes with fan (same as 38)
fileName        = 'antSpec42.npy'
spec42  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']


#short, circulator port 3 terminator via patch pannel
fileName        = 'antSpec43.npy'
spec43  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#same, warm term
fileName        = 'antSpec44.npy'
spec44  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#same, cool for 10 minutes with fan. Should be same as 43
fileName        = 'antSpec45.npy'
spec45  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W']

#short, circulator. port 1,2,3 = chode (short on other end), amp , noise source via patch
# use 28dB enr source + 12dB attenuator to get VERY hot terminator on patch pannel. 
#add 16.2dB attenuation before teledyne to prevent clipping!!! Correct for this here
# NS off
fileName        = 'antSpec46.npy'
spec46  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#NS on
fileName        = 'antSpec47.npy'
spec47  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#vivaldi, NS off
fileName        = 'antSpec48.npy'
spec48  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#vivaldi, NS on
fileName        = 'antSpec49.npy'
spec49  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#vivaldi, NS off (test after removing loads from room to give to Amin)
fileName        = 'antSpec50.npy'
spec50  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#vivaldi, NS on (test after removing loads from room to give to Amin)
#Different when only moving small metal loads from room. So sensitive!
# Coming back to this after scan 89/90. No difference, something else moved here
fileName        = 'antSpec51.npy'
spec51  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!


#NS -> HP step att @ 12 + MC 6dB + MC 7dB + MC 12dB = 37dB 
# call it NS + "37dB att"
fileName        = 'antSpec52.npy'
spec52  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#NS + "32dB att"
fileName        = 'antSpec53.npy'
spec53  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#NS + "29dB att"
fileName        = 'antSpec54.npy'
spec54  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#NS + "28dB att"
fileName        = 'antSpec55.npy'
spec55  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#NS + "27dB att"
fileName        = 'antSpec56.npy'
spec56  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#NS + "26dB att"
fileName        = 'antSpec57.npy'
spec57  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#NS + "25dB att (MC 12+6+7, step att = 0dB)"
fileName        = 'antSpec58.npy'
spec58  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#NS + "25dB att (remove MC 12dB, step att = 12dB)"
fileName        = 'antSpec59.npy'
spec59  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#NS + "24dB att (13dB MC, step att = 11dB)"
fileName        = 'antSpec60.npy'
spec60  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#NS + "23dB att (13dB MC, step att = 10dB)"
fileName        = 'antSpec61.npy'
spec61  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#NS + "22dB att (13dB MC, step att = 9dB)"
fileName        = 'antSpec62.npy'
spec62  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#NS + "21dB att (13dB MC, step att = 8dB)"
fileName        = 'antSpec63.npy'
spec63  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#NS + "20dB att (13dB MC, step att = 7dB)"
fileName        = 'antSpec64.npy'
spec64  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#NS + "17dB att (13dB MC, step att = 4dB)"
fileName        = 'antSpec65.npy'
spec65  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#NS + "13dB att (13dB MC, step att = 0dB)"
fileName        = 'antSpec66.npy'
spec66  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#NS + "10dB att (0dB MC, step att = 10dB)"
fileName        = 'antSpec67.npy'
spec67  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#NS + "0dB att (0dB MC, step att = 0dB)"
fileName        = 'antSpec68.npy'
spec68  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#sig gen. Sweep 400-500MHz -80dBm into port 3 of circulator via patch pannel
fileName        = 'antSpec69.npy'
spec69  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#sig gen. Sweep 400-500MHz -73dBm into port 3 of circulator via patch pannel
fileName        = 'antSpec70.npy'
spec70  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#sig gen. Sweep 400-500MHz -73dBm into port 3 of circulator via patch pannel
fileName        = 'antSpec71.npy'
spec71  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#siglent noise gen. 82mV into port 3 of circulator via patch pannel
fileName        = 'antSpec72.npy'
spec72  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#siglent noise gen. 82mV into port 3 of circulator via patch pannel
fileName        = 'antSpec73.npy'
spec73  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#pasternack term at LN2
fileName        = 'antSpec74.npy'
spec74  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#same, let it sit for 20 minutes longer. pasternack term at LN2
#looks the same as 74, which means term stops cooling off in ~10 minutes
fileName        = 'antSpec75.npy'
spec75  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#Short on port 3. Look at amp back action.
fileName        = 'antSpec76.npy'
spec76  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#open on port 3. Look at amp back action.
fileName        = 'antSpec77.npy'
spec77  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#Load on port 3. Look at amp back action.
fileName        = 'antSpec78.npy'
spec78  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#Amp (pe15a1012) input on port 3. Look at amp back action.
fileName        = 'antSpec79.npy'
spec79  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#Amp (pe15a1012) input on port 3. Look at amp back action.
fileName        = 'antSpec80.npy'
spec80  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#cryo term cable RT 200000 avg
fileName        = 'antSpec81.npy'
spec81  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#cryo term cable cold 0.2M avg
fileName        = 'antSpec82.npy'
spec82  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#cryo term cable cold 2M avg
fileName        = 'antSpec83.npy'
spec83  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#28dB NS, 3+12+7+6dB = 28 att 2M avg
fileName        = 'antSpec84.npy'
spec84  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#28dB NS, 4+12+7+6dB = 29 att 2M avg
fileName        = 'antSpec85.npy'
spec85  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#cryo cable room temp term
fileName        = 'antSpec86.npy'
spec86  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#pe15a1012 off output connected to circulator port 3 via patch pannel
fileName        = 'antSpec87.npy'
spec87  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#50ohm load (from VNA). Just confirm above isn't caused by drift
fileName        = 'antSpec88.npy'
spec88  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#place sma load on box. Term on port 3 via patch. 40k avg
fileName        = 'antSpec89.npy'
spec89  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#place sma load on box. 28dBatt NS on port 3 via patch. 40k avg
fileName        = 'antSpec90.npy'
spec90  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#place foil Z on box. 28dBatt NS on port 3 via patch. 40k avg
fileName        = 'antSpec91.npy'
spec91  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#place foil Z on box. load on port 3 via patch. 40k avg
#Looks like it messes with match and resonance seperately
fileName        = 'antSpec92.npy'
spec92  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#remove circulator. Try to keep things unmoved by placing it on top of amp with cable going to patch.
#this is the normal experiment (no switch)
fileName        = 'antSpec93.npy'
spec93  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

###10/2/23. Didn't touch antenna.
#remove pasternack amp and 16.2dB attenuation
fileName        = 'antSpec94.npy'
spec94  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(40/10) # attenuation corrected!

###10/5/23. Loads on amp, no circulator no cable
# 50ohm on amp  
fileName        = 'antSpec95.npy'
spec95  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(0/10) # no attenuation corrected!

# short on amp  
fileName        = 'antSpec96.npy'
spec96  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(0/10) # no attenuation corrected!

# open on amp  
fileName        = 'antSpec97.npy'
spec97  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(0/10) # no attenuation corrected!

# 1012-G input on front end amp input, power off  
fileName        = 'antSpec98.npy'
spec98  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(0/10) # no attenuation corrected!

# 1012-G input on front end amp input, power on, 12.0V 130mA  
fileName        = 'antSpec99.npy'
spec99  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(0/10) # no attenuation corrected!


# 1014 input on front end amp input, power off
fileName        = 'antSpec100.npy'
spec100  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(0/10) # no attenuation corrected!


# 1014 input on front end amp input, power on 12.0V 150mA
fileName        = 'antSpec101.npy'
spec101  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(0/10) # no attenuation corrected!

# MC zx60-A input on front end amp input, power off
fileName        = 'antSpec102.npy'
spec102  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(0/10) # no attenuation corrected!

#  MC zx60-A input on front end amp input, power on 12.0V 70mA
fileName        = 'antSpec103.npy'
spec103  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(0/10) # no attenuation corrected!

#create cable resonators
# circ [18inch cable -> short, amp, 50ohms]
fileName        = 'antSpec104.npy'
spec104  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(0/10) # no attenuation corrected!

# circ [24inch cable -> open, amp, 50ohms]
fileName        = 'antSpec105.npy'
spec105  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(0/10) # no attenuation corrected!

# 24inch cable -> open no circ.
fileName        = 'antSpec106.npy'
spec106  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(0/10) # no attenuation corrected!

# 6inch cable -> open no circ.
fileName        = 'antSpec107.npy'
spec107  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(0/10) # no attenuation corrected!

# 18 inch cable -> open no circ.
fileName        = 'antSpec108.npy'
spec108  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(0/10) # no attenuation corrected!


# circ [6inch cable -> open, amp, 50ohms]
fileName        = 'antSpec109.npy'
spec109  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(0/10) # no attenuation corrected!

# circ [6ft cable -> open, amp, 50ohms]
fileName        = 'antSpec110.npy'
spec110  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(0/10) # no attenuation corrected!

# circ [6ft cable -> short, amp, 50ohms]
fileName        = 'antSpec111.npy'
spec111  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(0/10) # no attenuation corrected!

#6ft cable -> short (no circ)
fileName        = 'antSpec112.npy'
spec112  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(0/10) # no attenuation corrected!

#6ft cable -> open (no circ)
fileName        = 'antSpec113.npy'
spec113  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(0/10) # no attenuation corrected!

#zx60 amp. 6ft cable -> short (no circ). 16dB att
fileName        = 'antSpec114.npy'
spec114  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(2.2/10) # no attenuation corrected!

#zx60 amp. 6ft cable -> 50ohms (no circ). 16dB att
fileName        = 'antSpec115.npy'
spec115  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(2.2/10) # no attenuation corrected!

###
#notice that antenna is higher than terminator. Suspect difference of set up. 
#retest with 16dB attenuation, everything constant

#vivaldi, chode, 1012
fileName        = 'antSpec116.npy'
spec116  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#term, chode, 1012
fileName        = 'antSpec117.npy'
spec117  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#term, chode, circ [chode, amp, term], 1012
fileName        = 'antSpec118.npy'
spec118  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#vivaldi, chode, circ [chode, amp, term], 1012
fileName        = 'antSpec119.npy'
spec119  = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#vivaldi, chode, circ [chode, amp, term], 1012 (100k avg)
fileName        = 'antSpec120.npy'
spec120 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#port 3 to term on patch pannel. Keep fan on term. Cool for 5 minutes
#vivaldi, chode, circ [chode, amp, term], 1012 (100k avg)
fileName        = 'antSpec121.npy'
spec121 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

## 10_11_23
# suspect amp is warm and putting out thermal noise as RF into the room. move it outside. 

#port 3 to term on patch pannel. turn amp on right before taking data (still in room)
#vivaldi, chode, circ [chode, amp, term], 1012 (10k avg)
fileName        = 'antSpec122.npy'
spec122 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#port 3 to term on patch pannel. Amp outside room. Same chain, just with both amps out of room
#vivaldi, chode, circ [chode, amp, term], 1012 (10k avg)
fileName        = 'antSpec123.npy'
spec123 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

# same as above, light bulb on
fileName        = 'antSpec124.npy'
spec124 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

# same as above, light bulb on, 100k avg
fileName        = 'antSpec125.npy'
spec125 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

# same as above, light bulb off, 100k avg
fileName        = 'antSpec126.npy'
spec126 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

# same amp chain/circulator as above, terminator 100k avg
fileName        = 'antSpec127.npy'
spec127 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

# same as 125, remove glass cover. 100k avg
fileName        = 'antSpec128.npy'
spec128 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

# same as above. Circulator has short on port 3 (on patch pannel via cable)
fileName        = 'antSpec129.npy'
spec129 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

# same as above. Circulator has hot source (29dB attenuation) on port 3 (on patch pannel via cable)
fileName        = 'antSpec130.npy'
spec130 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

##10_12_23. Closing in on write up. 100k avg, 2^18pt fft
# Term on circ, no chode. both amps out of room, circ in room. [term, patch pannel (to 1012), noise source OFF on patch pannel].
fileName        = 'antSpec131.npy'
spec131 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!


# Term on patch pannel (no circ.), no chode. both amps out of room
fileName        = 'antSpec132.npy'
spec132 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#10_13_23 back to messing around before write up
# vivaldi, no circ. both amps out of room
fileName        = 'antSpec133.npy'
spec133 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

# vivaldi, circ [chode, patch, NS off)] fan on
fileName        = 'antSpec134.npy'
spec134 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

# vivaldi, circ [chode, patch, NS off)] fan off
fileName        = 'antSpec135.npy'
spec135 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

# TEC cold, 1A. vivaldi, circ [chode, patch, NS off)],
fileName        = 'antSpec136.npy'
spec136 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

# TEC cold, residual (fan on, current off, psu on). vivaldi, circ [chode, patch, NS off)],
fileName        = 'antSpec137.npy'
spec137 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

# TEC cold, residual (fan on, current off, psu off). vivaldi, circ [chode, patch, NS off)],
fileName        = 'antSpec139.npy'
spec139 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

# same as before, repeat
fileName        = 'antSpec139.npy'
spec139 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#forget TEC
# cup of ice water on floor vivaldi, circ [chode, patch, NS off)],
fileName        = 'antSpec140.npy'
spec140 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

# heat sync in ice water on floor. vivaldi, circ [chode, patch, NS off)],
fileName        = 'antSpec141.npy'
spec141 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

# repeat 141 2 minutes later (temp may be changing still)
fileName        = 'antSpec142.npy'
spec142 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

# repeat 141 20 minutes later (temp may be changing still)
fileName        = 'antSpec143.npy'
spec143 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

# same as before, but also have a heat pack with heat sync on top.
fileName        = 'antSpec144.npy'
spec144 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

# repeat 144 2 minutes later
fileName        = 'antSpec145.npy'
spec145 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

# repeat 144. New heat pack, 2 heat syncs, 3 minutes to warm up
fileName        = 'antSpec146.npy'
spec146 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

# remove  heat pack and heat syncs
fileName        = 'antSpec147.npy'
spec147 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#preheat heat syncs with heat gun. 130c, but a few minutes to cool. 60C 5 minutes later, so likley 120-100C at time of test. 
fileName        = 'antSpec148.npy'
spec148 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#second 1012 amp in the room, in and out terminated. On. 
#power using interlock
fileName        = 'antSpec149.npy'
spec149 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#second 1012 amp in the room, in and out terminated. Off
#disconnect battery from interlock
fileName        = 'antSpec150.npy'
spec150 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#dummy amp on. Power resistor (8 ohms with 12V battery)
fileName        = 'antSpec151.npy'
spec151 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#dummy amp off. Power resistor (8 ohms with 12V battery)
fileName        = 'antSpec152.npy'
spec152 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#dummy amp on, with pb acid. Power resistor (8 ohms with 12V battery)
fileName        = 'antSpec153.npy'
spec153 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#ln2 dewar in room, nothing in it.
fileName        = 'antSpec154.npy'
spec154 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#omite50ohm dummy amp connected to interlock
fileName        = 'antSpec155.npy'
spec155 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#omite50ohm dummy amp directly to pb acid battery (no interlock)
fileName        = 'antSpec156.npy'
spec156 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#omite50ohm dummy amp connected to interlock add stool to mode stir
fileName        = 'antSpec157.npy'
spec157 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#same as 155, check. omite50ohm dummy amp connected to interlock, remove stool 
fileName        = 'antSpec158.npy'
spec158 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#same as 158, check. omite50ohm dummy amp connected to interlock, add stool
fileName        = 'antSpec159.npy'
spec159 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!


#same as 158, check. omite50ohm dummy amp connected to interlock, remove stool
fileName        = 'antSpec160.npy'
spec160 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#no interlock, 70W light on
fileName        = 'antSpec161.npy'
spec161 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#sight switch on, light unscrewed
fileName        = 'antSpec162.npy'
spec162 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#300W light on
fileName        = 'antSpec163.npy'
spec163 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

#70W light on
fileName        = 'antSpec164.npy'
spec164 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

### Ok doing the write up for real. The amp chain was dissambled for some unrelated noise figure measurement, so these may not agree with previous scans
### All amp stuff out of room. chain: patch, 1012, 1r5+, qt. 4 MC gold attenuators, 10, 6, 3, 1 dB, pink cable, MC silver 1dB att. 1200+ filter, teledyne
### 100k avg. 
#term on patch
fileName        = 'antSpec165.npy'
spec164 = np.load('./data_gitignore/' + fileName, allow_pickle = True).item()['chBSpec_W'] * 10**(16.2/10) # attenuation corrected!

import pandas as pd
from scipy.interpolate import interp1d

s11_df          = pd.read_csv('./data_gitignore/s11_9_29_23.csv')

# Interpolating s11 values
s11_interp_function = interp1d(s11_df['Frequency']/1e6, s11_df['S11_Magnitude (linear)'], kind='linear', fill_value="extrapolate")
s11_interp = s11_interp_function(freqs)


plt.close('all')
plt.figure()
#plt.plot(freqs[:-100], 10*np.log10(1000*spec1[1:-100]), label = 'nothing')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec2[1:-100]), label = 'bump dipole')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec3[1:-100]), label = 'open close door')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec4[1:-100]), label = 'turn attenuator')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec5[1:-100]), label = 'open vivaldi')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec6[1:-100]), label = 'term vivaldi')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec7[1:-100]), label = 'touch vivaldi')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec8[1:-100]), label = 'bump vivaldi')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec9[1:-100]), label = 'term off vivaldi')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec10[1:-100]), label = 'vivaldi behind')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec11[1:-100]), label = 'terminate vivaldi')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec12[1:-100]), label = 'bicon')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec13[1:-100]), label = 'yagi')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec14[1:-100]), label = 'term')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec15[1:-100]), label = 'open')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec16[1:-100]), label = 'short')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec17[1:-100]), label = 'term+LPF')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec18[1:-100]), label = 'short+turnaround')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec19[1:-100]), label = 'open+turnaround')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec20[1:-100]), label = 'term+LPF+1dB att')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec21[1:-100]), label = 'term+LPF+2dB att')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec22[1:-100]), label = 'big vivalidi')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec23[1:-100]), label = 'add open dipole')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec24[1:-100]), label = 'term dipole')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec25[1:-100]), label = 'vivaldi corner')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec26[1:-100]), label = 'vivaldi across room')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec27[1:-100]), label = 'across room w foam')


###9/27/23 play with circulator 

###vivaldi
#plt.plot(freqs[:-100], 10*np.log10(1000*spec28[1:-100]), label = 'dip open')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec29[1:-100]), label = 'vivaldi no circulator')

###loads
#plt.plot(freqs[:-100], 10*np.log10(1000*spec30[1:-100]), label = 'term + att')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec31[1:-100]), label = 'term')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec32[1:-100]), label = 'open')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec33[1:-100]), label = 'short')

###add circulator 
#plt.plot(freqs[:-100], 10*np.log10(1000*spec34[1:-100]), label = 'short + circulator')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec35[1:-100]), label = 'short + circulator (port 3 open)')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec36[1:-100]), label = 'short + circulator on amp (port 3 term)')

###circulator and vivaldi
#plt.plot(freqs[:-100], 10*np.log10(1000*spec37[1:-100]), label = 'vivaldi + circulator')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec38[1:-100]), label = 'vivaldi + circulator, move dipole')

###vivaldi, connect port 3 of circulator to termanator on patch pannel
#plt.plot(freqs[:-100], 10*np.log10(1000*spec39[1:-100]), label = 'vivaldi + circulator (ext term)')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec40[1:-100]), label = 'vivaldi + circulator (ext open)')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec41[1:-100]), label = 'vivaldi + circulator (ext WARM term)')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec42[1:-100]), label = 'vivaldi + circulator (ext cool term)')

###short. connect port 3 to patch pannel (like above)
#plt.plot(freqs[:-100], 10*np.log10(1000*spec43[1:-100]), label = 'short + circulator')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec44[1:-100]), label = 'short + circulator, warm term')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec45[1:-100]), label = 'short + circulator (10min cool)')

#same as above, but use NS on patch pannel
#plt.plot(freqs[:-100], 10*np.log10(1000*spec46[1:-100]), label = 'NS off')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec47[1:-100]), label = 'NS on')

#vivaldi, NS on patch pannel
#plt.plot(freqs[:-100], 10*np.log10(1000*spec48[1:-100]), label = 'NS off')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec49[1:-100]), label = 'NS on')
#take lunch, remove loads for Amin
#plt.plot(freqs[:-100], 10*np.log10(1000*spec50[1:-100]), label = 'NS off')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec51[1:-100]), label = 'NS on')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec52[1:-100]), label = 'NS-37dB')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec53[1:-100]), label = 'NS-32dB')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec54[1:-100]), label = 'NS-29dB')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec55[1:-100]), label = 'NS-28dB')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec56[1:-100]), label = 'NS-27dB')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec57[1:-100]), label = 'NS-26dB')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec58[1:-100]), label = 'NS-25dB')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec59[1:-100]), label = 'NS-25dB')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec60[1:-100]), label = 'NS-24dB')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec61[1:-100]), label = 'NS-23dB')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec62[1:-100]), label = 'NS-22dB')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec63[1:-100]), label = 'NS-21dB')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec64[1:-100]), label = 'NS-20dB')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec65[1:-100]), label = 'NS-17dB')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec66[1:-100]), label = 'NS-13dB')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec67[1:-100]), label = 'NS-10dB')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec68[1:-100]), label = 'NS-0dB')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec69[1:-100]), label = 'sig gen')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec70[1:-100]), label = 'sig gen')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec71[1:-100]), label = 'sig gen')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec72[1:-100]), label = 'noise gen 82mV')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec73[1:-100]), label = 'noise gen 22mV')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec74[1:-100]), label = 'term at ln2')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec75[1:-100]), label = 'term at ln2 longer')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec76[1:-100]), label = 'short')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec77[1:-100]), label = 'open')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec78[1:-100]), label = 'load')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec79[1:-100]), label = 'amp on')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec80[1:-100]), label = 'amp off')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec81[1:-100]), label = 'term RT .2M avg')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec82[1:-100]), label = 'term cold .02M avg')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec83[1:-100]), label = 'term cold 1M avg')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec84[1:-100]), label = 'term hot (28dBatt) 1M avg')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec85[1:-100]), label = 'term hot (29dB att) 1M avg')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec86[1:-100]), label = 'RT 1M avg')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec87[1:-100]), label = 'amp output')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec88[1:-100]), label = 'load')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec89[1:-100]), label = 'load, term on box')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec90[1:-100]), label = 'NS 29dB att, term on box')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec91[1:-100]), label = 'NS 29dB att, Z on box')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec92[1:-100]), label = 'load, Z on box')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec93[1:-100]), label = 'no circulator')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec94[1:-100]), label = 'only MC amp')
### loads on amp, no cable no circ. 10_5_23
#plt.plot(freqs[:-100], 10*np.log10(1000*spec95[1:-100]), label = 'term.')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec96[1:-100]), label = 'short')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec97[1:-100]), label = 'open')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec98[1:-100]), label = 'PNK 1012G off')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec99[1:-100]), label = 'PNK 1012G, on')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec100[1:-100]), label = 'PNK 1014, off')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec101[1:-100]), label = 'PNK 1014, on')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec102[1:-100]), label = 'MC zx60, off')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec103[1:-100]), label = 'MC zx60, on')
###cable resonator
#plt.plot(freqs[:-100], 10*np.log10(1000*spec104[1:-100]), label = '18 inch')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec105[1:-100]), label = '24 inch')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec106[1:-100]), label = '24 inch open, no circ.')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec107[1:-100]), label = '6 inch, no circ.')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec108[1:-100]), label = '18 inch, no circ.')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec109[1:-100]), label = '6 inch')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec110[1:-100]), label = '6 ft open')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec111[1:-100]), label = '6 ft short')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec112[1:-100]), label = '6 ft open, no circ')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec113[1:-100]), label = '6 ft short, no circ')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec114[1:-100]), label = '6 ft short, no circ, zx60')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec115[1:-100]), label = '6 ft term, no circ, zx60')

#10_10_23: testing why antenna is above term
plt.plot(freqs[:-100], 10*np.log10(1000*spec116[1:-100]), label = 'vivaldi')
plt.plot(freqs[:-100], 10*np.log10(1000*spec117[1:-100]), label = 'term')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec118[1:-100]), label = 'term')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec119[1:-100]), label = 'vivaldi')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec120[1:-100]), label = 'vivaldi')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec121[1:-100]), label = 'vivaldi, term on patch')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec122[1:-100]), label = 'vivaldi, term on patch, cold')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec123[1:-100]), label = 'vivaldi, term on patch, cold')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec124[1:-100]), label = 'light on')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec125[1:-100]), label = 'light on, 100k avg')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec126[1:-100]), label = 'light off, 100k avg')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec127[1:-100]), label = 'term, 100k avg')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec128[1:-100]), label = 'light on, no cover')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec129[1:-100]), label = 'light off, open on circ.')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec130[1:-100]), label = 'light off, hot on circ.')

#10_12_23 closing in on writeup. 10k avg
#plt.plot(freqs[:-100], 10*np.log10(1000*spec131[1:-100]), label = 'term on circ')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec132[1:-100]), label = 'term on patch')
plt.plot(freqs[:-100], 10*np.log10(1000*spec133[1:-100]), label = 'vivaldi, amps outside')

#messing around with temperture setting device  
#plt.plot(freqs[:-100], 10*np.log10(1000*spec135[1:-100]), label = 'heat sync fan off')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec134[1:-100]), label = 'heat sync fan on')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec136[1:-100]), label = 'TEC 1A, cold')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec137[1:-100]), label = 'TEC off, psu on, cold')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec139[1:-100]), label = 'TEC off, psu off cold')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec140[1:-100]), label = 'ice water')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec141[1:-100]), label = 'heat sync in ice water')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec142[1:-100]), label = 'heat sync in ice water, longer')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec143[1:-100]), label = 'heat sync in ice water, even longer')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec144[1:-100]), label = 'heat pack, 1 minute')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec145[1:-100]), label = 'heat pack, 3 minutes')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec146[1:-100]), label = 'new heat pack, 3 minutes')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec147[1:-100]), label = 'remove heat pack')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec148[1:-100]), label = '2 130C heat syncs')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec149[1:-100]), label = 'disconneced amp, on')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec150[1:-100]), label = 'disconneced amp, off')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec151[1:-100]), label = 'dummy amp, on')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec152[1:-100]), label = 'dummy amp, off')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec153[1:-100]), label = 'dummy amp, pb battery')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec154[1:-100]), label = 'no dummy amp, LN2 dewar in room')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec155[1:-100]), label = 'ohmite dummy amp interlock')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec156[1:-100]), label = 'ohmite dummy amp, no interlock')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec157[1:-100]), label = 'ohmite dummy amp, interlock, add stool')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec158[1:-100]), label = 'ohmite dummy amp, interlock, remove stool')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec159[1:-100]), label = 'ohmite dummy amp, interlock, add stool')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec160[1:-100]), label = 'ohmite dummy amp, interlock, remove stool')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec161[1:-100]), label = '70W light on') #no interlock, switch to light stuff
#plt.plot(freqs[:-100], 10*np.log10(1000*spec162[1:-100]), label = 'light switch on, light unscrewed')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec163[1:-100]), label = '300W light on')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec164[1:-100]), label = '70W light on')

plt.grid(True)
plt.legend(loc='upper right')
plt.xlabel('Freq. (MHz)')
plt.ylabel('Power (dBm)')




'''
print(f'mean 1: {spec1[startIdx:stopIdx].mean()}')
print(f'mean 2: {spec2[startIdx:stopIdx].mean()}')
print(f'mean 3: {spec3[startIdx:stopIdx].mean()}')
print(f'mean 4: {spec4[startIdx:stopIdx].mean()}')
print(f'mean 5: {spec5[startIdx:stopIdx].mean()}')
print(f'mean 6: {spec6[startIdx:stopIdx].mean()}')
print(f'mean 7: {spec7[startIdx:stopIdx].mean()}')
print(f'mean 8: {spec8[startIdx:stopIdx].mean()}')
print(f'mean 9: {spec9[startIdx:stopIdx].mean()}')
print(f'mean 10: {spec10[startIdx:stopIdx].mean()}')
print(f'mean 11: {spec11[startIdx:stopIdx].mean()}')
print(f'mean 12: {spec12[startIdx:stopIdx].mean()}')
print(f'mean 13: {spec13[startIdx:stopIdx].mean()}')
print(f'mean 14: {spec14[startIdx:stopIdx].mean()}')
print(f'mean 15: {spec15[startIdx:stopIdx].mean()}')
print(f'mean 16: {spec16[startIdx:stopIdx].mean()}')
print(f'mean 17: {spec17[startIdx:stopIdx].mean()}')
print(f'mean 25: {spec25[startIdx:stopIdx].mean()}')
print(f'mean 26: {spec26[startIdx:stopIdx].mean()}')
print(f'mean 27: {spec27[startIdx:stopIdx].mean()}')'''

In [ ]:
#Calibration


plt.close('all')
plt.figure()
plt.tight_layout()
plt.title('Output Referred Power Spectral Density')
plt.plot(freqs[:-100], spec132[1:-100]*1e9, label = f'50$\Omega$ terminator')
plt.plot(freqs[:-100], (spec132[1:-100]-2.07e-10)*1e9, label = f'50$\Omega$ terminator, corrected for NF')
plt.xlabel('Frequency (MHz)')
plt.ylabel('Power (nW/9.5kHz)')
plt.legend()
plt.grid(True)
plt.xlim(10, 1000)
plt.ylim(.2, 2)

termNorm = (spec132[1:-100]-2.07e-10)

In [ ]:
#coax resonator
plt.close('all')
plt.figure()
plt.title('Coax Resonator')
plt.plot(freqs[:-100], spec95[1:-100]/(spec132[1:-100]), label = f'50$\Omega$ terminator')
plt.plot(freqs[:-100], spec113[1:-100]/(spec132[1:-100]), label = 'Short on 1.8m cable')
plt.xlabel('Frequency (MHz)')
plt.ylabel('Power (dBm)')
plt.legend()
plt.grid(True)
#plt.xlim(10, 1000)
#plt.ylim(-67, -57)

#add markers
# Define your list of specific marker frequencies
marker_freqs = [28, 82, 136.7, 464.8]  # Replace with your list of desired frequencies
display_offset = [.5, -1.5, 0, 0]

for i in range(len(marker_freqs)):
    # Find the corresponding power value for the marker frequency from your data
    power_at_marker_freq113 = 10*np.log10(1000*spec113[np.argmin(np.abs(freqs - marker_freqs[i]))])

    # Add a marker for spec95
    plt.scatter(marker_freqs[i], power_at_marker_freq113, color='red', marker='^', s=100, zorder=5)  # Adjust color, marker style, size (s=...) as needed
    #plt.text(marker_freqs[i], power_at_marker_freq113 + display_offset[i], f'{marker_freqs[i]} MHz', fontsize=13, verticalalignment='bottom')



plt.show()

In [ ]:
#note switch is hanging on amp from 28-35, past that circulator is on amp

###loads
plt.close('all')
plt.figure()
plt.title('Loads directly into amp')
#plt.plot(freqs[:-100], 10*np.log10(1000*spec30[1:-100]), label = 'term + att')
plt.plot(freqs[:-100], 10*np.log10(1000*spec31[1:-100]), label = 'term')
plt.plot(freqs[:-100], 10*np.log10(1000*spec32[1:-100]), label = 'open')
plt.plot(freqs[:-100], 10*np.log10(1000*spec33[1:-100]), label = 'short')
plt.grid(True)
plt.legend(loc='upper left')
#plt.xlim(450,550)
plt.ylim(-65,-58)
plt.ylabel('Power(dBm)')
plt.xlabel('freq (MHz)')
plt.show()

###add circulator. 34, 35 have it on antenna side of chode cable.  
#34: circulator. port 1,2,3 = short, chode cable, terminator.
#35: circulator. port 1,2,3 = short, chode cable, open.
#36: hang citculator on amp. Remove swich (was used in 28-35)
#### circulator. port 1,2,3 = chode (short on other end), amp , term.

plt.figure()
plt.title('Loads with circulator, ports [1,2,3] \n terminator for refrence')
plt.plot(freqs[:-100], 10*np.log10(1000*spec31[1:-100]), label = 'no circ! term, cable, amp')
plt.plot(freqs[:-100], 10*np.log10(1000*spec34[1:-100]), label = 'circ. on load [short, cable (to amp), term.]')
plt.plot(freqs[:-100], 10*np.log10(1000*spec36[1:-100]), label = 'circ. on amp [cable (short on other end), amp , term.]')
plt.plot(freqs[:-100], 10*np.log10(1000*spec35[1:-100]), label = 'circ. on load [short, cable (to amp), open]')
plt.xlim(450,550)
plt.ylim(-64,-58)
plt.grid(True)
plt.legend(loc='upper left')
plt.ylabel('Power(dBm)')
plt.xlabel('freq (MHz)')


plt.figure()
plt.title('short driect and short on circulator')
plt.plot(freqs[:-100], 10*np.log10(1000*spec33[1:-100]), label = 'short')
plt.plot(freqs[:-100], 10*np.log10(1000*spec32[1:-100]), label = 'open')
plt.plot(freqs[:-100], 10*np.log10(1000*spec35[1:-100]), label = 'circ. on load [short, cable (to amp), open]')
plt.xlim(400,600)
plt.ylim(-65,-60)
plt.grid(True)
plt.legend(loc='upper left')
plt.ylabel('Power(dBm)')
plt.xlabel('freq (MHz)')

In [ ]:
M = 1 - s11_interp**2

plt.close('all')
plt.figure()
plt.plot(freqs, M, label = '1-s11^2')
plt.plot(freqs, s11_interp, label = 's11')
plt.plot(freqs[:-100], (spec72[1:-100])/spec72[idxStart:idxStop].mean(), label = 'noise gen/mean')
plt.legend(loc = 'upper right')
plt.xlim(400, 600)
plt.ylim(-.3,1.3)

plt.figure()
plt.plot(freqs, (spec72[1:])/M, label = 'noise gen/M')
plt.plot(freqs, (spec72[1:])*M, label = 'noise gen*M')
plt.legend(loc = 'upper right')

In [ ]:

plt.close('all')
M = 1 - s11_interp**2
idxStart = int(2**17 * 475/1250)
idxStop = int(2**17 * 525/1250)

plt.figure()
plt.title('M = 1-s11^2 and power spec (normalized by mean)')
plt.plot(freqs, M, label = '1-s11^2')
plt.plot(freqs[:-100], (spec76[1:-100])/spec76[idxStart:idxStop].mean(), label = 'open circ. port 3')
plt.plot(freqs[:-100], (spec93[1:-100])/spec93[idxStart:idxStop].mean(), label = 'no circ.')
plt.plot(freqs[:-100], (spec79[1:-100])/spec79[idxStart:idxStop].mean(), label = 'amp input via circ.')
plt.legend(loc='upper left')
plt.grid(True)
plt.ylabel('Power and M (linear, dimentionless)')
plt.xlabel('freq (MHz)')
plt.xlim(400, 600)
plt.ylim(.5,1.5)


plt.figure()
plt.title('power spec of different loads on port 3 of circulator')
plt.plot(freqs[:-100], (spec76[1:-100]), label = 'open')
plt.plot(freqs[:-100], (spec86[1:-100]), label = 'room temp load')
plt.plot(freqs[:-100], (spec83[1:-100]), label = 'cold load')
plt.plot(freqs[:-100], (spec84[1:-100]), label = 'hot load')
plt.plot(freqs, M/1e9, label = 'M/10^9')
plt.plot(freqs, (spec31[1:]), label = 'terminator into amp')
#plt.plot(freqs[:-100], (spec93[1:-100])/spec93[idxStart:idxStop].mean(), label = 'no circ.')
#plt.plot(freqs[:-100], (spec79[1:-100])/spec79[idxStart:idxStop].mean(), label = 'amp input via circ.')
plt.legend(loc='upper left')
plt.grid(True)
plt.ylabel('Power and M (linear, dimentionless)')
plt.xlabel('freq (MHz)')
plt.xlim(400, 600)
plt.ylim(.5e-9,1.3e-9)


plt.figure()
plt.title('power spec of different loads on port 3 of circulator vs straight term.')
plt.plot(freqs[:-100], (spec86[1:-100]), label = 'room temp load')
plt.plot(freqs[:-100], (spec84[1:-100]), label = 'hot load')
plt.plot(freqs[:-100], (spec83[1:-100]), label = 'cold load')
plt.plot(freqs, (spec31[1:]), label = 'terminator into amp', alpha = 0.5)
#plt.plot(freqs[:-100], (spec93[1:-100])/spec93[idxStart:idxStop].mean(), label = 'no circ.')
#plt.plot(freqs[:-100], (spec79[1:-100])/spec79[idxStart:idxStop].mean(), label = 'amp input via circ.')
plt.legend(loc='upper left')
plt.grid(True)
plt.ylabel('Power and M (linear, dimentionless)')
plt.xlabel('freq (MHz)')
plt.xlim(400, 600)
plt.ylim(.5e-9,1.3e-9)



print(len(M))
plt.figure()
plt.title('power spec of different loads on port 3 of circulator/M')
#plt.plot(s11_df['Frequency']/1e6, M, label = '1-s11^2')
#plt.plot(freqs[:-100], (spec76[1:-100]), label = 'open')
plt.plot(freqs, (spec86[1:])/M, label = 'room temp load/M')
plt.plot(freqs, (spec83[1:])/M, label = 'cold load/M')
plt.plot(freqs, (spec79[1:])/M, label = 'amp load/M')
plt.plot(freqs, (spec84[1:])/M, label = 'hot load/M')
plt.plot(freqs, (spec93[1:])/M, label = 'no circ./M')
plt.plot(freqs, (spec63[1:])/M, label = 'very hot load/M')
#plt.plot(freqs, M, label = 'hot load')
#plt.plot(freqs[:-100], (spec93[1:-100])/spec93[idxStart:idxStop].mean(), label = 'no circ.')
#plt.plot(freqs[:-100], (spec79[1:-100])/spec79[idxStart:idxStop].mean(), label = 'amp input via circ.')
plt.legend(loc='upper left')
plt.grid(True)
plt.ylabel('Power/M (linear)')
plt.xlabel('freq (MHz)')
plt.xlim(400, 600)
plt.ylim(5e-10, 2.3e-9)



In [ ]:
plt.close('all')

plt.figure()
plt.title('power spec of different loads/M')
plt.plot(freqs, M, label = '1-s11^2')
#plt.plot(freqs[:-100], (spec86[1:-100])/spec86[idxStart:idxStop].mean(), label = 'room temp load, with circ.')
#plt.plot(freqs[:-100], (spec83[1:-100])/spec83[idxStart:idxStop].mean(), label = 'cold load, with circ')
#plt.plot(freqs[:-100], (spec84[1:-100])/spec84[idxStart:idxStop].mean(), label = 'hot load, with circ.')
plt.plot(freqs[:-100], (spec76[1:-100])/spec76[idxStart:idxStop].mean(), label = 'open circ. port 3')
plt.plot(freqs[:-100], (spec93[1:-100])/spec93[idxStart:idxStop].mean(), label = 'no circ.')
plt.plot(freqs[:-100], (spec79[1:-100])/spec79[idxStart:idxStop].mean(), label = 'amp input via circ.')
plt.legend()
plt.grid(True)
plt.ylabel('Power and M (linear, dimentionless)')
plt.xlabel('freq (MHz)')
plt.xlim(400, 600)
plt.ylim(.5,1.5)

In [ ]:


plt.figure()
plt.title('1-s11^2 and power spec (normalized by mean)')
plt.plot(s11_df['Frequency']/1e6, M, label = '1-s11^2')
plt.plot(freqs[:-100], (spec86[1:-100])/spec86[idxStart:idxStop].mean(), label = 'room temp load, with circ.')
plt.plot(freqs[:-100], (spec83[1:-100])/spec83[idxStart:idxStop].mean(), label = 'cold load, with circ')
plt.plot(freqs[:-100], (spec84[1:-100])/spec84[idxStart:idxStop].mean(), label = 'hot load, with circ.')
plt.plot(freqs[:-100], (spec93[1:-100])/spec93[idxStart:idxStop].mean(), label = 'no circ.')
plt.plot(freqs[:-100], (spec79[1:-100])/spec79[idxStart:idxStop].mean(), label = 'amp input via circ.')
plt.legend()
plt.grid(True)
plt.ylabel('Power(dBm)')
plt.xlabel('freq (MHz)')

plt.figure()
plt.title('1-s11^2')
plt.plot(s11_df['Frequency']/1e6, M, label = '1-s11^2')
plt.plot(freqs[:-100], (spec83[1:-100])/spec83[idxStart:idxStop].mean(), label = 'cold load, with circ. (normilized by mean)')
plt.plot(freqs[:-100], (spec93[1:-100])/spec93[idxStart:idxStop].mean(), label = 'no circ.')
plt.legend()
plt.grid(True)
plt.ylabel('Power(dBm)')
plt.xlabel('freq (MHz)')

# Model for what we are seeing related to an observable
$T, T_H, T_C:$ Room temp, T_H, T_C, where $T-T_H = T-T_C$. In other words $T_C=77$ K, so $T_H=290+(290-77k)\approx500K$ 

$P^{O}_{T} = $ output referred power with terminator at T 

$P^{O}_{T} = G(N_R*M+N_A)$ where G is gain, N_R is the noise in the room (per freq), N_A is the amp noise (per freq)

$N_A = kT + <Q>kT(T_t/T - 1)$ where T_t is temp of terminator and $<Q>$ I think is how exciteable the room is

skipping a bunch of algebra:
$\frac{P^O_H + P^O_C }{P^O} = 2$


In [ ]:
#first pass, less averaging
termRT = spec50[1:-100] #room temp
termHot = spec54[1:-100] #hot
termCold = spec74[1:-100] #cold


#1M average
termRT = spec86[1:-100] #room temp
termHot = spec85[1:-100] #hot
termCold = spec83[1:-100] #cold


plt.figure()
plt.plot(freqs[:-100], termCold/termRT, label = 'cold/RT')
plt.plot(freqs[:-100], termHot/termRT, label = 'hot/RT')
plt.xlim(350, 650)
plt.ylim(.7,1.3)
plt.ylabel('linear ratio')
plt.xlabel('freq (MHz)')
plt.legend()
plt.grid(True)
plt.show()


plt.figure()
plt.plot(freqs[:-100], ((termCold+termHot)/termRT), label = '(hot+cold)/room temp')
plt.xlim(350, 650)
plt.ylim(1.925,2.025)
plt.ylabel('linear ratio')
plt.xlabel('freq (MHz)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
#similar to above, but move front amp to outside of room and don't normilize
plt.close('all')

plt.figure()
plt.plot(freqs[:-100], spec126[1:-100], label = 'light off, term on circ')
plt.plot(freqs[:-100], spec127[1:-100], label = 'term')
plt.plot(freqs[:-100], spec128[1:-100], label = 'light on')
plt.plot(freqs[:-100], spec129[1:-100], label = 'light off, open on circ.')
plt.plot(freqs[:-100], spec130[1:-100], label = 'light off, hot on circ.')
plt.legend()
plt.xlim(450, 550)
plt.ylim(0.73e-9, 1.1e-9)
plt.grid(True)


plt.figure()
plt.plot(freqs[:-100], spec129[1:-100] + spec130[1:-100], label = '"cold" + hot')
plt.xlim(450, 550)
plt.ylim(1.75e-9, 2.1e-9)

plt.figure()
plt.plot(freqs[:-100], (spec129[1:-100] + spec130[1:-100])/spec127[1:-100], label = '("cold" + hot)/room temp')
plt.xlim(450, 550)

In [ ]:
# the interlock radiates broadband noise which interacts with the room and is modified by "stirrer" (a metal stool)

plt.close('all')
plt.figure()
plt.plot(freqs[:-100], 10*np.log10(1000*spec158[1:-100]), label = 'ohmite dummy amp, interlock, remove stool')
plt.plot(freqs[:-100], 10*np.log10(1000*spec159[1:-100]), 'g', label = 'ohmite dummy amp, interlock, add stool')
plt.plot(freqs[:-100], 10*np.log10(1000*spec160[1:-100]), 'r', label = 'ohmite dummy amp, interlock, remove stool')
plt.legend()
plt.grid(True)